<a href="https://colab.research.google.com/github/ichekhovskikh/fasttext/blob/master/RecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk
from gensim.models.fasttext import FastText

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem

#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели FastText. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 500 научных статей на 5 различных тем.

Для тестирования будет использоваться тестовый корпус из 50 статей (10 статей на каждую тему).

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать), class_name (был размечен вручную, необходим только для тестирования)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/train_corpus.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/cyberleninka-article-downloader/master/test_corpus.json' #@param {type: "string"}

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        for article in corpus:
            normalized_text = advanced_preprocess(article['text'])
            yield normalized_text

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Получение категории статьи по индексу:

In [0]:
def get_article_class_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['class_name']

Загружаем корпуса:

In [0]:
train_corpus = list(read_corpus(train_path))
test_corpus = list(read_corpus(test_path))

Давайте посмотрим на учебный и тестовый корпуса:

In [0]:
print(train_corpus[:2])

Корпус тестирования выглядит так:

In [0]:
print(test_corpus[:2])

# Обучение модели
## Создание объекта FastText
Теперь мы создадим модель FastText с векторным размером 80 слов и перебираем учебный корпус 100 раз.
Параметры обучения:
- size: размерность вектора
- window: «окно», в котором рассматривается контекст
- alpha: коэфициент обучения
- sg: cbow (0) или skip-gram (1)
- epochs: количество иттераций

In [0]:
#@title Укажите параметры обучения модели:
vector_size = 80 #@param
window =  8 #@param
epochs =  100 #@param
alpha = 0.001 #@param
learning_method = "skip-gram" #@param ["skip-gram", "CBOW"] {type:"string"}

sg = 1
if (learning_method == "CBOW"):
    sg = 0

In [0]:
model = FastText(
    size=vector_size, 
    window=window,
    alpha=alpha,
    sg=sg)
model.build_vocab(sentences=train_corpus)
%time model.train(sentences=train_corpus, total_examples=model.corpus_count, epochs=epochs)

# Опредлим функции для преобразования вектора слов в вектор текста

Обратная сглаженная частота (вес слова), где
- a: гиперпараметр, который обычно равен 0.001 (конфигурация, внешняя по отношению к модели, значение которой невозможно оценить по данным):

In [0]:
a = 0.001

def sif(word, paragraph):
    frequency = paragraph.count(word) / len(paragraph)
    return a / (a + frequency) 
    

Получить вектор текста:

In [0]:
def paragraph_vector(model, paragraph):
    result_vector = []
    for word in paragraph:
        weight = sif(word, paragraph)
        try:
            word_vector = model.wv[word]
            weighted_word_vector = [weight * elem for elem in word_vector]
            if not result_vector:
                result_vector = weighted_word_vector
            else: 
                result_vector = [x + y for x, y in zip(result_vector, weighted_word_vector)]
        except:
            continue
    return result_vector

Вычислим вектора для всех статей исходной базы для ускорения работы:

In [0]:
paragraph_vectors = []
for text in train_corpus:
    text_vector = paragraph_vector(model, text)
    paragraph_vectors.append(text_vector)

# Поиск похожих текстов на основе обученной модели

Определение косинусной близости двух векторов:

In [0]:
def cosine_similarity(first_vector, second_vector):
    numerator = sum([x * y for x, y in zip(first_vector, second_vector)])
    first_norm = pow(sum([x * x for x in first_vector]), 0.5)
    second_norm = pow(sum([x * x for x in second_vector]), 0.5)
    return numerator / (first_norm * second_norm)

Поиск похожих текстов из корпуса

In [0]:
def sims(model, paragraph, count = 5):
    result = []
    compared_vector = paragraph_vector(model, paragraph)
    for index, text_vector in enumerate(paragraph_vectors):
        similarity = cosine_similarity(compared_vector, text_vector)
        result.append([index, similarity])
    return sorted(result,  key=lambda x: x[1], reverse=True)[:count]

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
doc_id = random.randint(0, len(train_corpus) - 1)
doc = train_corpus[doc_id]
sims = sims(model, doc)
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, train_path)))
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
from sklearn.metrics import f1_score

test_size = len(test_corpus)

predicted_classes = [] 
test_classes = []

for doc_index in range(test_size):
    doc = test_corpus[doc_index]
    similarities = sims(model, doc)
    
    test_article_class = get_article_class_by_index(doc_index, test_path)
    print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}) «{}»: «{}»\n'.format(doc_index, test_article_class, get_article_text_by_index(doc_index, test_path)))
    num = 0
    for index, similarity in similarities:
        num += 1
        predicted_article_class = get_article_class_by_index(index, train_path)
        print(u'%s) %s «%s»: «%s»\n' % (num, similarity, predicted_article_class, get_article_text_by_index(index, train_path)))
        predicted_classes.append(predicted_article_class)
        test_classes.append(test_article_class)
        
print('f1score: {}'.format(f1_score(test_classes, predicted_classes, average='macro')))

Тестирование на малых данных для выявления ошибок в ходе написания кода

In [0]:
from gensim.test.utils import common_texts

print(common_texts)

model = FastText(size=80, window=4, min_count=2, sample=1e-2, sg=1)
model.build_vocab(sentences=common_texts)
model.train(sentences=common_texts, total_examples=model.corpus_count, epochs=10)

first_vector = paragraph_vector(model, ['human', 'interface', 'computer'])
second_vector = paragraph_vector(model, ['human', 'user', 'interface', 'system'])
print(cosine_similarity(first_vector, second_vector))

# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read().decode()


In [0]:
normalized_text = advanced_preprocess(article_text)
sims = sims(model, normalized_text)

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА «{}»\n'.format(article_text)
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))